##### Let's do the branch and bound problem

In [82]:
from numpy import matmul
from itertools import product
from math import sqrt, floor

In [83]:
constraint_value = 68644
const_coeff = [9, 8, 7, 7, 6, 6, 5, 2, 1, 1]
org_coeff = [104, 128, 135, 139, 150, 153, 162, 168, 195, 198]

If we set every other value (say Xj) to 0, how large does Xi (i != j) have to be to meet this constraint?
How many possibilities would we have to explore? 

In [84]:
upper_bounds = []
product = 1

for i in range(10):
    val = floor(sqrt(constraint_value / const_coeff[i]))
    product *= val
    upper_bounds.append(val)

for _, val in enumerate(upper_bounds):
    print(f"The upper bound of x_{_} is {val}")
print(f"\nWe'd have to explore {product} possibilities in total.")

The upper bound of x_0 is 87
The upper bound of x_1 is 92
The upper bound of x_2 is 99
The upper bound of x_3 is 99
The upper bound of x_4 is 106
The upper bound of x_5 is 106
The upper bound of x_6 is 117
The upper bound of x_7 is 185
The upper bound of x_8 is 262
The upper bound of x_9 is 262

We'd have to explore 1309632284192829030720 possibilities in total.


That's too many possibilities. We're going to use branch and bound to try and do better.

In [85]:
# we already have the list of upper bounds
# we need the lower bounds
def calculate_u(root, alpha, beta):
    u = sum([(a ** 2 / b) for a, b in zip(alpha, beta)])
    return sqrt(u) / (2 * root)

def coeff(u, found_values=[]):
    c = [(i / (2 * j * u)) for i, j in zip(org_coeff[:10-len(found_values)], const_coeff[:10-len(found_values)])]
    return c

def sol_val(x):
    return matmul(org_coeff, x)

lower_bounds = []

u = calculate_u(sqrt(constraint_value), org_coeff, const_coeff)
x_values = [int(i) for i in coeff(u)]
lower_bounds = sol_val(x_values)

In [86]:
def actual_lb(x):
    curr_r_val = constraint_value - sum([(i * j * j) for i, j in zip(const_coeff[::-1], x)])
    
    if curr_r_val < 0:
        return 0
    elif curr_r_val == 0:
        return sol_val([0] * (10 - len(x)) + x)
    
    u = calculate_u(sqrt(curr_r_val), org_coeff[:10-len(x)], const_coeff[:10-len(x)])
    x_values = coeff(u, x)
    return sol_val(x_values + x[::-1])

In [87]:
count = 0
solution = []

for x10 in range(upper_bounds[9], -1, -1):
  if actual_lb(x=[x10]) >= lower_bounds:
      for x9 in range(upper_bounds[8], -1, -1):
        if actual_lb(x=[x10,x9]) >= lower_bounds:
            for x8 in range(upper_bounds[7], -1, -1):
              if actual_lb(x=[x10,x9,x8]) >= lower_bounds:
                  for x7 in range(upper_bounds[6], -1, -1):
                    if actual_lb(x=[x10,x9,x8,x7]) >= lower_bounds:      
                        for x6 in range(upper_bounds[5], -1, -1):
                          if actual_lb(x=[x10,x9,x8,x7,x6]) >= lower_bounds:                         
                              for x5 in range(upper_bounds[4], -1, -1):
                                if actual_lb(x=[x10,x9,x8,x7,x6,x5]) >= lower_bounds:                                
                                    for x4 in range(upper_bounds[3], -1, -1):
                                      if actual_lb(x=[x10, x9, x8, x7, x6, x5, x4]) >= lower_bounds:                                     
                                          for x3 in range(upper_bounds[2], -1, -1):
                                            if actual_lb(x=[x10,x9,x8,x7,x6,x5, x4, x3]) >= lower_bounds:                                              
                                                for x2 in range(upper_bounds[1], -1, -1):
                                                  if actual_lb(x=[x10,x9,x8,x7,x6,x5, x4, x3, x2]) >=lower_bounds:                                                   
                                                      for x1 in range(upper_bounds[0], -1, -1):
                                                        if actual_lb(x=[x10,x9,x8,x7,x6, x5, x4, x3, x2,x1]) >= lower_bounds:                                                         
                                                            x = [x1, x2, x3, x4, x5,x6,x7,x8,x9,x10] 
                                                            if sol_val(x) >lower_bounds: 
                                                              lower_bounds = sol_val(x) 
                                                              solution.clear() 
                                                              solution.append(x)
                                                            elif sol_val(x) == lower_bounds:                                
                                                              solution.append(x)
                                                            count += 1

print(f"Maximum value: {lower_bounds}")
print(f"Complete variable assignments visited: {count}")
print(f"Optimum: {solution}")

TypeError: 'int' object is not callable